In [ ]:
# 包引入和全局变量定义模块
import pandas as pd
from jqdata import *
import seaborn as sns
from datetime import datetime,timedelta,date
import numpy as np
import matplotlib.pyplot as plt
import json
import math
import random
from pyfinance import TSeries as tss
from functools import reduce


sns.set_style("whitegrid",{"font.sans-serif":['simhei','Droid Sans Fallback']})
pd.set_option('display.max_rows',100)
pd.set_option('display.max_columns',None)



In [ ]:
class Position(object):
    def __init__(self, security, price, total_amount, closeable_amount, avg_cost):
        self.security = security
        self.price = price
        self.total_amount = total_amount
        self.closeable_amount = closeable_amount
        self.avg_cost = avg_cost

    def __str__(self):
        return str(self.__dict__)

    @property
    def locked_amount(self):
        return self.total_amount - self.closeable_amount

    @property
    def value(self):
        return self.total_amount * self.price

    def get_dict_data(self):
        origin_dict = self.__dict__
        origin_dict['value'] = self.value
        return origin_dict

    def position_add(self, price_now, now_amount):
        avg_cost = (self.avg_cost * self.total_amount + price_now * now_amount) / (self.total_amount + now_amount)
        self.avg_cost = avg_cost
        self.total_amount += now_amount

    def position_minus(self, now_amount):
        minus_amount = min(self.closeable_amount, now_amount)
        self.total_amount -= minus_amount
        self.closeable_amount -= minus_amount



class Portfolio(object):
    def __init__(self, init_cash):
        self.inout_cash = init_cash
        self.available_cash = init_cash
        self.positions = {}

    @property
    def positions_value(self):
        sum_value = 0
        for stock, stock_info in self.positions.items():
            sum_value += stock_info.value
        return sum_value

    @property
    def total_value(self):
        return self.positions_value + self.available_cash

    def __str__(self):
        return json.dumps(self, default=lambda o: o.__dict__,
                          sort_keys=True, indent=4)


class G(object):
    def __init__(self):
        self.hold_info = pd.DataFrame.from_dict(
            {
                'security': [], 'price': [], 'total_amount': [],
                'closeable_amount': [], 'avg_cost': [], 'value': [], 'end_date': []}
        )

    def clean(self):
        self.hold_info = pd.DataFrame.from_dict(
            {
                'security': [], 'price': [], 'total_amount': [],
                'closeable_amount': [], 'avg_cost': [], 'value': [], 'end_date': []}
        )


class Context(object):
    def __init__(self,init_cash,date_from,date_to):
        self.init_cash = init_cash
        self.date_from = date_from
        self.date_to = date_to
        self.portfolio = Portfolio(init_cash)
        self.dict_func = {}


g = G()



In [ ]:
# 通用函数定义



def util_fund_portfolio(end_date, list_code=None,pd_fund_origin=None):
    dict_fund_pub_date = {'第四季度': ('12-31', '01-31'), '第一季度': ('03-31', '04-30'),
                          '第二季度': ('06-30', '07-31'), '第三季度': ('09-30', '10-31')}

    if pd_fund_origin is not None:
        max_date = pd_fund_origin['end_date'].max()
        if max_date>=end_date:
            return pd_fund_origin

    if list_code is None:
        list_code = get_all_securities(types=['stock'], date=end_date).index.tolist()

    pd_code = pd.DataFrame.from_dict({'code':list(set(list_code))})

    result = []


    if pd_fund_origin is None:
        init_date = (datetime.strptime(end_date,'%Y-%m-%d')-timedelta(days=750)).strftime('%Y-%m-%d')
        start_year = int(init_date.split('-')[0])
        end_year = int(end_date.split('-')[0])
        for i in range(end_year - start_year + 1):
            year_iter = start_year + i

            for key, value in dict_fund_pub_date.items():

                if key == '第四季度' or key == '年度':
                    year_real_start = str(year_iter - 1) + '-' + value[0]
                else:
                    year_real_start = str(year_iter) + '-' + value[0]

                year_real_end = str(year_iter) + '-' + value[1]

                if datetime.strptime(year_real_start, '%Y-%m-%d').date() > datetime.strptime(end_date,
                                                                                             '%Y-%m-%d').date():
                    break

                if datetime.strptime(year_real_end, '%Y-%m-%d').date() > datetime.strptime(end_date, '%Y-%m-%d').date():
                    year_real_end = end_date

#                 print(year_real_start + '----' + year_real_end+':'+'util_fund_portfolio')

                fund_portfolio = util_get_fund_portfolio_by_chunks(year_real_start, year_real_end)
                result.append(fund_portfolio)
        if len(result) != 0:
            pd_result = pd.concat(result)
        else:
            raise ValueError()
    else:
        init_date = (datetime.strptime(end_date,'%Y-%m-%d')-timedelta(days=180)).strftime('%Y-%m-%d')
        year_real_start = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=30)).strftime('%Y-%m-%d')
        year_real_end = end_date
#         print(year_real_start + '----' + year_real_end+':'+'util_fund_portfolio')
        pd_result_one = util_get_fund_portfolio_by_chunks(year_real_start, year_real_end)
        if pd_result_one is not None:
            pd_result = pd.concat([pd_fund_origin, pd_result_one],  ignore_index=True,sort=True)
        else:
            return pd_fund_origin

    pd_result = pd_result[pd_result['end_date']>init_date]
    pd_result = pd.merge(pd_code,pd_result,on=['code'])
    return pd_result


def util_turnOver_monthly(end_date,list_code = None,pd_result_origin=None,num_month=4):

    diff_days = 100

    if pd_result_origin is not None:
        max_date = pd_result_origin['end_date'].max()
        diff_days = int((datetime.strptime(end_date,'%Y-%m-%d') - datetime.strptime(max_date,'%Y-%m-%d')).days)

    if diff_days<=5:
        return pd_result_origin

    if list_code is None:
        list_code_tmp = get_all_securities(types=['stock'], date=end_date).index.tolist()
    else:
        list_code_tmp = list_code

    pd_result = util_run_func_by_chunk(list_code_tmp, int(10000/diff_days), get_valuation, end_date=end_date,
                                             count=diff_days,
                                             fields=['turnover_ratio'])

    pd_result['flag_date'] = pd_result['day'].map(lambda x:'-'.join(x.strftime('%Y-%m-%d').split('-')[0:2]))
    pd_result = pd_result.rename(columns = {'turnover_ratio':'turnOver_monthly'})
    pd_result = pd_result.groupby(['code','flag_date'])['turnOver_monthly'].mean().reset_index()
    pd_result['end_date'] = end_date

    pd_result_final = pd.concat([pd_result_origin,pd_result],ignore_index=True,sort=True).drop_duplicates(subset=['code','flag_date'],keep='first')
    pd_result_final = pd_result_final.sort_values(by=['code','flag_date'],ascending=False).groupby(['code']).head(num_month).sort_values(by=['code','flag_date'])

    return pd_result_final


def util_turnOver_days(end_date,list_code = None,pd_result_origin=None,interval=20,num=4):

    diff_days = interval*num-1

    if pd_result_origin is not None:
        max_date = pd_result_origin['end_date'].max()
        if max_date>=end_date:
            return pd_result_origin

    if list_code is None:
        list_code_tmp = get_all_securities(types=['stock'], date=end_date).index.tolist()
    else:
        list_code_tmp = list_code

    pd_result = util_run_func_by_chunk(list_code_tmp, int(10000/diff_days), get_valuation, end_date=end_date,
                                             count=diff_days,
                                             fields=['turnover_ratio'])

    pd_result = pd_result.sort_values(by=['code','day'])
    pd_result['flag_num'] = pd_result.groupby(['code'])['turnover_ratio'].transform(lambda x: [int((i+1)/interval) for i in range(len(x))])
    pd_result = pd_result.rename(columns = {'turnover_ratio':'turnOver_monthly'})
    pd_result = pd_result.groupby(['code','flag_num'])['turnOver_monthly'].mean().reset_index()
    pd_result['end_date'] = end_date
    return pd_result


def util_get_month_K(end_date,list_code=None,count=90):
    if list_code is None:
        list_code = get_all_securities(types=['stock'], date=end_date).index.tolist()

    pd_result = get_price(list_code, count=count, end_date=end_date, frequency='daily', fields=['close','open','high','low'], panel=False, fill_paused=True)
    pd_result['end_date'] = pd_result['time'].map(lambda x:'-'.join(x.strftime('%Y-%m-%d').split('-')[0:2]))
    pd_result = pd_result.groupby(['code','end_date']).agg({'low':'min','high':'max','close':'last','open':'first'}).reset_index()

    return pd_result


def util_get_all_stock(end_date):
    stock_list = get_all_securities(types=['stock'], date=end_date).index.tolist()
    stock_list = util_filter_special(end_date,stock_list)
    return stock_list


def util_filter_special(end_date,stock_list):  # 过滤器，过滤停牌，ST，科创，新股
    df_extras = get_extras('is_st', stock_list, count=1, end_date=end_date)
    df_extras = df_extras.stack().reset_index().rename(columns = {'level_0':'end_date','level_1':'code',0:'is_st'})
    l_extras = df_extras[df_extras['is_st'].map(lambda x: not x)]['code'].values.tolist()
    pd_price = get_price(stock_list, count=1, end_date=end_date, frequency='daily', fields=['paused'],panel=False)
    l_price = pd_price[pd_price['paused']==0]['code'].values.tolist()
    set_stock = set(l_extras).intersection(set(l_price))
    return list(set_stock)



def util_get_fund_portfolio_by_chunks(start_date, end_date):
    """
    在start_date 到 end_date 日期内，list_stock里的股票被所有基金的持仓情况统计
    """

    def chunks(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    fund = get_all_securities(types=['open_fund'], date=end_date).index.tolist()
    list_fund = list(map(lambda x: x[0:6], fund))

    pd_result = pd.DataFrame.from_dict({
        'symbol': [], 'start_date': [], 'end_date': [], 'report_type': [], 'stock_count': [],
        'share_sum': [], 'market_cap_sum': []
    })

    list_all_stock_code = []

    for once_code in chunks(list_fund, 4000):
        q = query(finance.FUND_MAIN_INFO).filter(
            finance.FUND_MAIN_INFO.main_code.in_(once_code),
            finance.FUND_MAIN_INFO.underlying_asset_type_id.in_([402001, 402004]),
            finance.FUND_MAIN_INFO.operate_mode == '开放式基金')
        code_list = finance.run_query(q)['main_code'].unique().tolist()
        list_all_stock_code += set(code_list)


    for once_code in chunks(list_fund, 4000):
        q = query(finance.FUND_PORTFOLIO.code, finance.FUND_PORTFOLIO.total_asset).filter(
                finance.FUND_PORTFOLIO.code.in_(list_all_stock_code),
                finance.FUND_PORTFOLIO.pub_date <= end_date,
                finance.FUND_PORTFOLIO.pub_date >= start_date)
        fund_asset = finance.run_query(q)
        # 筛选出净值大约1个亿的基金，基金净值过小容易被清盘，风格漂移
        l_fund_asset_code = fund_asset[fund_asset['total_asset'] >= 100000000]['code'].unique().tolist()

    list_all_stock_code = list(set(list_all_stock_code).intersection(set(l_fund_asset_code)))

    total_fund = len(list_all_stock_code)

    for once_code in chunks(list_all_stock_code, 300):
        q = query(finance.FUND_PORTFOLIO_STOCK).filter(finance.FUND_PORTFOLIO_STOCK.code.in_(once_code),
                                                       finance.FUND_PORTFOLIO_STOCK.pub_date <= end_date,
                                                       finance.FUND_PORTFOLIO_STOCK.pub_date >= start_date,
                                                       finance.FUND_PORTFOLIO_STOCK.rank <= 10,
                                                       )

        fund_portfolio = finance.run_query(q)

        stock_als = fund_portfolio.groupby(['symbol', 'report_type']).agg(
            {'shares': 'sum', 'symbol': 'count', 'market_cap': 'sum'}).rename(
            columns={'symbol': 'stock_count', 'shares': 'share_sum', 'market_cap': 'market_cap_sum'}).reset_index()

        if len(stock_als) != 0:
            stock_als['start_date'] = start_date
            stock_als['end_date'] = end_date
            stock_als = stock_als[
                ['symbol', 'start_date', 'end_date', 'report_type', 'stock_count', 'share_sum', 'market_cap_sum']]
            pd_result = pd.concat([pd_result, stock_als]).groupby(
                ['symbol', 'start_date', 'end_date', 'report_type']).sum().reset_index()

    if len(pd_result)==0:
        return None
    pd_result['total_fund'] = total_fund
    pd_result = util_normalize_code(pd_result, 'symbol')
    pd_result = util_add_pub_year(pd_result)
    pd_result = pd_result[(pd_result['code'] != 'unknown') & (pd_result['pub_date'] != 'unknown')]
    pd_result = pd_result.sort_values(
        by=['code', 'start_date', 'end_date', 'report_type', 'stock_count']).drop_duplicates(
        subset=['code', 'start_date', 'end_date', 'report_type'], keep='last').drop(['symbol'], axis=1)
    pd_result['pub_date'] = pd_result['pub_date'].map(
        lambda x: x[0:10] if type(x) == str else x.strftime('%Y-%m-%d'))
    pd_result['stock_count'] = pd_result['stock_count'].map(lambda x: 10 if x < 10 else x)
    return pd_result



def util_run_func_by_chunk(list_code, n, func, **args):
    result = None
    def chunks(lst, n):
        for i in range(0, len(lst), n):
            yield lst[i:i + n]
    for once_code in chunks(list_code, n):
        result = pd.concat([func(once_code, **args),result],ignore_index=True)
    return result


def util_last_day_of_month(any_day):
    import datetime
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)


def util_normalize_code(pd_data, code_col_name='code'):
    """
    通过股票code 得到股票名称
    """
    l_code_o = pd_data[code_col_name].values.tolist()
    l_norm_code = []

    for one_code in l_code_o:
        try:
            l_norm_code.append(normalize_code(one_code))
        except:
            l_norm_code.append('unknown')

    pd_data['code'] = l_norm_code
    return pd_data


def util_add_pub_year(pd_data, code_col_name='code'):
    """
    通过股票code 得到股票名称
    """

    l_code_o = pd_data[code_col_name].values.tolist()
    l_name = []

    for one_code in l_code_o:
        try:
            l_name.append(get_security_info(one_code).start_date)
        except:
            l_name.append('unknown')

    pd_data['pub_date'] = l_name
    return pd_data


def util_add_name(pd_data, code_col_name='code'):
    """
    通过股票code 得到股票名称
    """

    l_code_o = pd_data[code_col_name].values.tolist()
    l_name = []

    for one_code in l_code_o:
        try:
            l_name.append(get_security_info(one_code).display_name)
        except:
            l_name.append('未知')

    pd_data['display_name'] = l_name
    return pd_data


def util_get_price_now(security, end_date, time_str):
    pd_result = get_price(security, count=1, end_date=end_date + ' ' + time_str, frequency='1m',
                          fields=['close', 'open'], panel=False, fill_paused=True)
    if len(pd_result) == 0:
        return None
    price_now = pd_result['open'].values[0]
    return price_now


def util_get_paused_now(security, end_date, time_str):
    pd_result = get_price(security, count=1, end_date=end_date + ' ' + time_str, frequency='1m',
                          fields=['paused'], panel=False, fill_paused=True)
    if len(pd_result) == 0:
        return None
    paused_now = pd_result['paused'].values[0]
    return paused_now


def util_with_weight_money(money_available, num):
    a = np.linspace(1, 10, num)
    theta = money_available / sum(a)
    b = a * theta
    return b.astype(int)


def util_split_current_day(dt):
    dt_str_end = dt.strftime('%Y-%m-%d %H:%M:%S')
    one_day = dt_str_end.split(' ')[0]
    time_str = dt_str_end.split(' ')[1]
    return one_day,time_str



def util_maxDrawdown(return_list):
    i = np.argmax((np.maximum.accumulate(return_list) - return_list) / np.maximum.accumulate(return_list))  # 结束位置
    if i == 0:
        return 0
    j = np.argmax(return_list[:i])  # 开始位置
    return (return_list[j] - return_list[i]) / (return_list[j])


In [ ]:
# 因子生成函数定义


def cond_turnOver_30(data_o, N):
    def cond_turnOver_30_inner(data_grp_o):
        threshold = 0
        x = data_grp_o.values[-4:]
        result = 0
        for i in range(len(x)):
            one_x = x[i]
            if threshold >= N:
                result=1
            else:
                result=0
            if 0.3 < one_x < 0.8:
                threshold += 1
            else:
                threshold = 0
        return result

    pd_result = data_o.groupby(['code'])['turnOver_monthly'].agg(cond_turnOver_30_inner).reset_index().rename(
        columns={'turnOver_monthly': 'cond_turnOver_30'})
    pd_result = pd_result[pd_result['cond_turnOver_30'] > 0]
    return pd_result['code'].unique().tolist()



def alpha_turnOver_30(data_o):
    pd_result = data_o.sort_values(by=['code','flag_date']).drop_duplicates(subset=['code'],keep='last')
    pd_result.rename(columns={'turnOver_monthly': 'alpha_turnOver_30'},inplace=True)
    return pd_result[['code','alpha_turnOver_30']].drop_duplicates()



def cond_plus_10(data_o, N=4):
    data_o['flag'] = data_o['stock_count'].map(lambda x: 1 if x > 10 else 0)
    data_o['cond_plus_10'] = data_o.groupby('code')['flag'].cumsum()
    data_o = data_o[data_o['cond_plus_10'] >= N]
    return data_o['code'].unique().tolist()


def cond_pct_stock_count(data_o, thold=0.02):
    data_o = data_o.sort_values(by=['code', 'end_date'])
    data_o['cond_pct_stockCount'] = data_o.groupby(['code'])['stock_count'].pct_change()
    max_end_date = data_o['end_date'].values.max()
    data_o = data_o[data_o['end_date']==max_end_date]
    data_o = data_o[data_o['cond_pct_stockCount']>=thold]
    return data_o['code'].unique().tolist()


def alpha_pct_stock_count(data_o):
    data_o = data_o.sort_values(by=['code', 'end_date'])
    data_o['alpha_pct_stockCount'] = data_o.groupby(['code'])['stock_count'].pct_change()
    data_o['max_date'] = data_o.groupby(['code'])['end_date'].transform(max)
    max_end_date = data_o['end_date'].values.max()
    data_o = data_o[data_o['end_date']==max_end_date]
    return data_o[['code', 'alpha_pct_stockCount']].drop_duplicates()


def alpha_profit_month(data_p):
    def alpha_profit_month_inner(data_grp_o):
        price_all = data_grp_o['close'].values
        cum_profit_10 = []
        cum_profit_1 = []
        cum_profit_3 = []
        for i, one_price in enumerate(price_all):
            if i >= 1:
                cum_profit_1.append((one_price - price_all[0]) / price_all[0])
            else:
                cum_profit_1.append(0)
            if i >= 3:
                cum_profit_3.append(((one_price - price_all[i - 3]) / price_all[i - 3]))
            else:
                cum_profit_3.append(0)


        pd_result = pd.DataFrame.from_dict(
            {'cum_profit_1': cum_profit_1, 'cum_profit_3': cum_profit_3})
        pd_result['smart_price'] =  0.6 * pd_result['cum_profit_3'] + 0.4 * pd_result[
            'cum_profit_1']
        pd_result = pd_result.set_index(data_grp_o['end_date'])
        return pd_result



    r = data_p.groupby(['code']).apply(alpha_profit_month_inner).reset_index().drop_duplicates()
    r = r.rename(columns={'cum_profit_10': 'alpha_profit_month_10', 'cum_profit_1': 'alpha_profit_month_1',
                          'cum_profit_3': 'alpha_profit_month_3'})

    max_end_date = r['end_date'].values.max()
    r = r[r['end_date']==max_end_date]
    del r['end_date']

    return r.drop_duplicates()


In [ ]:
# 因子生成模块

def run_candidate_stock(context):
    dt_str_end = context.current_dt.date().strftime('%Y-%m-%d')
    g.candidate_list = util_get_all_stock(dt_str_end)


def run_cond_turnOver(context,stock_list=None):
    if stock_list is None:
        stock_list = g.candidate_list
    dt_str_end = context.current_dt.date().strftime('%Y-%m-%d')
    g.pd_turnOver_origin = util_turnOver_monthly(dt_str_end,stock_list,g.pd_turnOver_origin)
    return cond_turnOver_30(g.pd_turnOver_origin, 3)


def run_cond_pct_stockCount(context,stock_list=None):
    if stock_list is None:
        stock_list = g.candidate_list
    dt_str_end = context.current_dt.date().strftime('%Y-%m-%d')
    if context.current_dt.date().month in [1,4,7,10]:
        g.pd_fund_origin = util_fund_portfolio(dt_str_end,list_code=stock_list,pd_fund_origin=g.pd_fund_origin)
        return cond_pct_stock_count(g.pd_fund_origin,0.02)
    else:
        return stock_list


def run_cond_plus_10(context,stock_list=None):
    if stock_list is None:
        stock_list = g.candidate_list
    dt_str_end = context.current_dt.date().strftime('%Y-%m-%d')
    if context.current_dt.date().month in [1,4,7,10]:
        len_origin = len(g.cond_plus_10)
        g.pd_fund_origin = util_fund_portfolio(dt_str_end,list_code=stock_list,pd_fund_origin=g.pd_fund_origin)
        g.cond_plus_10 = list(set(cond_plus_10(g.pd_fund_origin)).union(set(g.cond_plus_10)))
    return g.cond_plus_10.copy()


def run_cond_clist(context,stock_list=None):
    dt_str_end = context.current_dt.date().strftime('%Y-%m-%d')
    if context.current_dt.date().month in [1,4,7,10]:
        return g.candidate_list
    else:
        return g.cond_list


def run_alpha_pct_stock_count(context):
    stock_list = g.cond_list
    pd_stock_filter = pd.DataFrame.from_dict({'code':stock_list})

    dt_str_end = context.current_dt.date().strftime('%Y-%m-%d')
    pd_fund_origin = util_fund_portfolio(dt_str_end,stock_list,g.pd_fund_origin)

    pd_fund_tmp = pd.merge(pd_stock_filter,pd_fund_origin)

    if len(pd_fund_tmp)!=0:
        pd_fund_tmp = alpha_pct_stock_count(pd_fund_tmp)
    return pd_fund_tmp


def run_alpha_profit_month(context):
    stock_list = g.cond_list
    pd_stock_filter = pd.DataFrame.from_dict({'code':stock_list})

    dt_str_end = context.current_dt.date().strftime('%Y-%m-%d')
    pd_price_all = util_get_month_K(dt_str_end,stock_list)

    pd_price_tmp = pd.merge(pd_stock_filter,pd_price_all)

    if len(pd_price_tmp)!=0:
        pd_price_tmp = alpha_profit_month(pd_price_tmp)
    return pd_price_tmp


def run_alpha_turnOver(context,stock_list=None):
    stock_list = g.cond_list

    pd_stock_filter = pd.DataFrame.from_dict({'code':stock_list})

    dt_str_end = context.current_dt.date().strftime('%Y-%m-%d')
    pd_turnOver_origin = util_turnOver_monthly(dt_str_end,stock_list,g.pd_turnOver_origin)

    pd_turnOver_tmp = pd.merge(pd_stock_filter,pd_turnOver_origin)

    if len(pd_turnOver_tmp)!=0:
        pd_turnOver_tmp = alpha_turnOver_30(pd_turnOver_tmp)
    return pd_turnOver_tmp


def run_cond_all(context):
    dt_str_end = context.current_dt.date().strftime('%Y-%m-%d')

    list_filter = []
    cond0 = set(run_cond_clist(context))

    list_filter = list(cond0)
    if len(list_filter)==0:
        g.cond_list = []
        return
    cond1 = set(run_cond_plus_10(context,list_filter))

    list_filter = list(set(list_filter).intersection(cond1))
    if len(list_filter)==0:
        g.cond_list = []
        return
    cond2 = set(run_cond_pct_stockCount(context,list_filter))

    list_filter = list(set(list_filter).intersection(cond2))
    if len(list_filter)==0:
        g.cond_list = []
        return
    cond3 = set(run_cond_turnOver(context,list_filter))


    list_filter = list(set(list_filter).intersection(cond3))
    g.cond_list = list_filter

    print(dt_str_end+'  '+'cond_list：',g.cond_list)


def run_alpha_all(context):
    if len(g.cond_list)==0:
        g.alpha_data = None
        return
    alpha1 = run_alpha_pct_stock_count(context)
    alpha2 = run_alpha_profit_month(context)
    alpha3 = run_alpha_turnOver(context)
    g.alpha_data = pd.merge(alpha1, alpha2, on=['code'], how='outer')
    g.alpha_data = pd.merge(g.alpha_data, alpha3, on=['code'], how='outer')


def run_close_day(context):

    result_now = []

    end_date,time_str = util_split_current_day(context.current_dt)
    positions = context.portfolio.positions
    for key, stock in positions.items():
        price_now = util_get_price_now(key, end_date, time_str)
        if price_now is not None:
            stock.price = price_now
        stock.closeable_amount = stock.total_amount
        result_now.append(stock.get_dict_data())
    if len(result_now)>0:
        pd_info = pd.DataFrame.from_dict(result_now)
        pd_info['end_date'] = end_date
        g.hold_info = pd.concat([g.hold_info,pd_info],ignore_index=True,sort=True)

    pd_cash = pd.DataFrame.from_dict(
            {
                'security': ['cash'], 'price': [-1], 'total_amount': [-1],
                'closeable_amount': [-1], 'avg_cost': [-1], 'value': [context.portfolio.available_cash], 'end_date': [end_date]}
        )

    g.hold_info = pd.concat([g.hold_info,pd_cash],ignore_index=True,sort=True)


In [ ]:
# 购买策略模块

def run_strategy(context):
    dt_str_end = context.current_dt.date().strftime('%Y-%m-%d')
    buy_strategy_base(context)
    print(dt_str_end+'  '+'buy_list：',g.buy_list)


def run_buy(context):
    buy_with_cash_weight(context)


def run_sell(context):
    sell_all(context)


def buy_strategy_base(context):

    rank_param = {'sort_1': {'by': ['alpha_pct_stockCount', 'alpha_profit_month_3', 'alpha_turnOver_30'],
                              'ascending': [False, False, True]}, 'N_1': 10,
                   'sort_2': {'by': ['alpha_profit_month_3', 'alpha_turnOver_30'],
                              'ascending': [True, True]}, 'N_2': 10}

    print('购买策略：buy_strategy_1')

    if g.alpha_data is None:
        g.sell_list = g.bought_list
        g.buy_list = list()
        g.bought_list = list()
    else:
        dt = context.current_dt.date()
        data_alpha = g.alpha_data
        if dt.month in [1,4,7,10]:
            data_o = data_alpha.fillna(0)
            code_now = data_o.sort_values(**rank_param.get('sort_1')).head(rank_param.get('N_1'))['code'].unique().tolist()
        else:
            data_o = data_alpha.fillna(10000000)
            code_now = data_o.sort_values(**rank_param.get('sort_2')).head(rank_param.get('N_2'))['code'].unique().tolist()
        g.buy_list = code_now
        g.sell_list = set(g.bought_list) - set(code_now)
        g.bought_list = []


def buy_with_cash_weight(context):
    if len(g.buy_list) == 0:
        return

    value_not_sell = 0
    for not_sell_s in g.bought_list:
        stock_info = context.portfolio.positions[not_sell_s]
        value_not_sell += stock_info.price * stock_info.total_amount

    available_value = context.portfolio.total_value - value_not_sell
#     available_value = context.portfolio.available_cash

    list_money = util_with_weight_money(available_value, len(g.buy_list))

    dict_buy = {}

    for i, one_stock in enumerate(g.buy_list):
        if context.portfolio.positions.get(one_stock) is not None:
            stock_info = context.portfolio.positions[one_stock]
            value_now = stock_info.price * stock_info.total_amount
        else:
            value_now = 0
        dict_buy[one_stock] = (list_money[i], value_now, list_money[i] - value_now)

    dict_buy = dict(sorted(dict_buy.items(), key=lambda item: item[1][2]))
    # 先卖后买
    for one_stock, value in dict_buy.items():
        result = order_target_value(context,one_stock, value[0])
        if result is None:
            print('WARN(买入{}失败,买入量：{}'.format(one_stock, str(value[0])))
            if context.portfolio.positions.get(one_stock) is not None:
                if context.portfolio.positions[one_stock].total_amount > 0:
                    g.bought_list.append(one_stock)
        else:
#             print('买入{}成功,价值为{}'.format(one_stock,context.portfolio.positions[one_stock].value))
            g.bought_list.append(one_stock)


def sell_all(context):
    print('卖出：' + ','.join(list(g.sell_list)))
    for one_stock in g.sell_list:
        result = order_target(context,one_stock, 0)
        if result is None:
            print('WARN(卖出失败):' + one_stock)
            g.bought_list.append(one_stock)




def update_all_price(context):
    end_date,time_str = util_split_current_day(context.current_dt)
    portfolio = context.portfolio
    for stock, stock_info in portfolio.positions.items():
        price_now = util_get_price_now(stock, end_date, time_str)
        if price_now is not None:
            stock_info.price = price_now


In [ ]:
# Order 模块

def order(context, stock, num):
    available_cash = context.portfolio.available_cash
    num_real = int(num / 100) * 100

    end_date,time_str = util_split_current_day(context.current_dt)

    portfolio = context.portfolio
    price_now = util_get_price_now(stock, end_date, time_str)

    value_need = price_now * num_real
    num_real = int(min(value_need,available_cash)/price_now/100)*100

    if num_real >= 100:
        context.portfolio.available_cash -= num_real*price_now
        if portfolio.positions.get(stock) is not None:
            portfolio.positions[stock].position_add(price_now, num_real)
        else:
            portfolio.positions[stock] = Position(stock, price_now, num_real, 0, price_now)
        if available_cash >= value_need:
            return True
        else:
            print('WARN：无足够资金购买：{}，所需资金为：{},可用资金为：{},调整为资金为：{}'.format(stock,value_need,available_cash,num_real*price_now))
            return True

    else:
        print('ERROR：购买{}数量低于100'.format(stock))
        return None


def sell(context, stock, num):
    num_real = int(num / 100) * 100

    end_date,time_str = util_split_current_day(context.current_dt)

    portfolio = context.portfolio

    paused = util_get_paused_now(stock, end_date, time_str)
    price_now = util_get_price_now(stock, end_date, time_str)

    if paused == 0:
        if num_real>=100:
            if portfolio.positions.get(stock) is not None:
                value_for = price_now * num_real
                portfolio.positions[stock].position_minus(num_real)
                context.portfolio.available_cash += value_for
                if portfolio.positions[stock].value==0:
                    context.portfolio.positions.pop(stock)
                elif portfolio.positions[stock].value<0:
                    raise ValueError()
                return True
            else:
                print('ERROR：当前标的{}无持仓'.format(stock))
                return None
        else:
            print('ERROR：出售{},数量低于100'.format(stock))
            return None
    else:
        print('WARN：当前标的{},暂停交易'.format(stock))
        return None



def order_target_value(context, stock, value):
    end_date,time_str = util_split_current_day(context.current_dt)
    price_now = util_get_price_now(stock, end_date, time_str)
    num_real = int(value / price_now / 100) * 100
    return order_target(context, stock, num_real)


def order_target(context, stock, num):
    portfolio = context.portfolio
    if portfolio.positions.get(stock) is not None:
        num_diff =  num - portfolio.positions[stock].closeable_amount
        if num_diff >= 100:
            return order(context, stock, num_diff)
        elif num_diff <= -100:
            return sell(context, stock, -num_diff)
        else:
            return True
    else:
        return order(context, stock, num)

In [ ]:
# back test 模块
def run_month(func,day_num,time,context,reference_security):
    list_trade_days = prepare_get_custom_trade_days(context,day_num)
    prepare_add_to_context(context,list_trade_days,time,func)


def run_daily(func,time,context,reference_security='000300.XSHG'):
    list_trade_days = get_trade_days(start_date=context.date_from, end_date=context.date_to)
    prepare_add_to_context(context,list_trade_days,time,func)


def prepare_get_custom_trade_days(context,day_num):
    list_trade_days = get_trade_days(start_date=context.date_from, end_date=context.date_to)
    pd_date = pd.DataFrame.from_dict({'trade_days':list_trade_days})
    pd_date['grp_flag'] = pd_date['trade_days'].map(lambda x: '-'.join(x.strftime('%Y-%m-%d').split('-')[0:2]))
    l_trade_real_date = pd_date.groupby(['grp_flag'])['trade_days'].nth(day_num).values.tolist()
    return l_trade_real_date


def prepare_add_to_context(context,list_trade_days,time,func):
    for one_day in list_trade_days:
        r_dict = {}
        if context.dict_func.get(one_day) is not None:
            r_dict = context.dict_func.get(one_day)
        r_list_func = []
        if r_dict.get(time) is not None:
            r_list_func = r_dict.get(time)
        r_list_func.append(func)
        r_dict[time] = r_list_func
        context.dict_func[one_day] = dict(sorted(r_dict.items(), key=lambda x:x[0]))


def dispatch_main(init_money,start_date,end_date):
    import time
    start_time = time.time()

    context = Context(init_money,start_date,end_date)
    g.clean()

    initialize(context)
    list_trade_days = get_trade_days(start_date=context.date_from, end_date=context.date_to)
    for one_day in list_trade_days:
        if context.dict_func.get(one_day) is not None:
            func_values = context.dict_func[one_day]
            for time_one,one_func_list in func_values.items():
                for one_func in one_func_list:
                    one_day_str = one_day.strftime('%Y-%m-%d')
                    if one_func.__name__!='run_close_day':
                        print(one_day_str+' '+time_one+' ',one_func.__name__)
                    update_context(context,one_day_str,time_one)
                    one_func(context)

    end_time = time.time()
    print('共耗时')
    print(end_time - start_time)


def update_context(context,one_day_str,time_str):
    context.current_dt = datetime.strptime(one_day_str+' '+time_str,'%Y-%m-%d %H:%M:%S')




In [ ]:
# 初始化函数，设定基准等等
def initialize(context):
    # 设定沪深300作为基准
    # 过滤掉order系列API产生的比error级别低的log
    g.index_security = '000300.XSHG'
    g.buy_list = []
    g.bought_list = []
    g.sell_list = []
    g.pd_fund_origin=None
    g.pd_turnOver_origin=None
    g.cond_plus_10 = []
    g.cond_list = []

    ### 股票相关设定 ###
    # 股票类每笔交易时的手续费是：买入时佣金万分之三，卖出时佣金万分之三加千分之一印花税, 每笔交易佣金最低扣5块钱
#     set_order_cost(OrderCost(close_tax=0.001, open_commission=0.0003, close_commission=0.0003, min_commission=5),
#                    type='stock')

    run_month(run_cond_all,-1,'08:00:00',context,'000300.XSHG')
    run_month(run_candidate_stock,-1,'07:00:00',context,'000300.XSHG')
    run_month(run_alpha_all,-1,'09:00:00',context,'000300.XSHG')
    run_month(run_strategy,-1,'09:00:00',context,'000300.XSHG')

    run_month(update_all_price,-1,'10:00:00',context,'000300.XSHG')
    run_month(run_sell,-1,'10:00:00',context,'000300.XSHG')

    run_month(update_all_price,-1,'14:00:00',context,'000300.XSHG')
    run_month(run_buy,-1,'14:00:00',context,'000300.XSHG')

    run_daily(run_close_day,'16:00:00',context,'000300.XSHG')




In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


dispatch_main(1000000,'2013-01-01','2021-03-01')
